# Bitcoin

En este ejercicio veremos los conceptos de bitcoin que están más relacionados con criptografía

In [ ]:
!python3 -m pip install pycryptodome

## Proof of work Hashcat

Vamos a programar una "Proof of work" (*pow*) muy sencilla inspirada en [Hashcat](https://en.wikipedia.org/wiki/Hashcash), la misma tecnología utilizada por bitcoin: "encuentra un número tal que al juntarlo a un texto y calcular su hash, el hash empiece por un número determinado de ceros".

La idea es que no se aceptará un correo electrónico, nuevo bloque bitcoin... lo que sea, si el emisor no incluye una "proof of work" junto con el mensaje. Es decir, que pruebe que ha estado X minutos buscando el contador que resulte en un hash correcto. De esta manera se limita el número de mensajes por minuto que puede emitir una persona. Fíjate que encontrar el contador es un proceso costoso, pero comprobar que el contador es correcto es muy rápido: solo tienes que calcular el hash y mirar si empieza con el número correcto de ceros.

Supongamos que el día 2023-01-02 a las 17:35 queremos enviar un mensaje a employee@mail.com. Para evitar spam, al sistema quiere asegurarse de que solo enviaremos un correo cada minuto, por ejemplo, y para eso estima que tiene que obligarnos a un esfuerzo "5". Para esto, necesitamos crear una "proof of work" que tiene el siguiente aspecto: `2301021735:employee@mail.com:CONTADOR`. El `CONTADOR` es lo que tenemos que encontrar: debemos probar contadores al azar hasta que los 5 primeros bytes (el esfuerzo) del hash SHA-256 sean 0

Una propuesta de función sería así:


In [ ]:
from Crypto.Hash import SHA256
import time

def proof_of_work(msg, effort):
    start = time.time()
    test = '0' * effort
    ctr = 0
    blk = f'{msg}:{ctr}'.encode()
    h = SHA256.new(data=blk).hexdigest()
    while not h[:effort] == test:
        ctr += 1
        blk = f'{msg}:{ctr}'.encode()
        h = SHA256.new(data=blk).hexdigest()
    seconds = time.time() - start
    return dict(proof=blk, hash=h, seconds=seconds)

Y vamos a probar con tres esfuerzos diferentes: que el hash `301021735:employee@mail.com:CONTADOR` empiece por 1, 4 ó 5 ceros. Esto es lo que se llama "minar" en bitcoin.

In [ ]:
msg = "2301021735:employee@mail.com"

print(proof_of_work(msg, effort=1))
print(proof_of_work(msg, effort=4))
print(proof_of_work(msg, effort=5))

Fíjate, en el último caso, calcular el contador le lleva medio minuto aproximadamente, pero probar que el contador es correcto es muy rápido. El servidor de correo no aceptará ningún correo si el emisor no ha calculado el contador. En cambio, comprobar si el emisor ha esperado esos segundos es algo rápido. Esta es la base de bitcoin: minar es lento, comprobar si el minado fue correcto es rápido.

In [ ]:
SHA256.new(data=b"2301021735:employee@mail.com:1300743").hexdigest()[0:5] == '00000'

## Transacciones en bitcoin

Una transacción en bitcoin incluye: (1) el identificador de la cartera que envía bitcoin, (2) el identificador de la cartera que los recibe, (3) la cantidad, (4) lo que se paga a los validadores, (5) la hora y (6) la firma digital de la persona que envía el dinero.

![](images/transaction.png)

Aquí vamos a usar identificadores de cartera cortos y aleatorios. En realidad, el identificador de una cartera se crea a partir de la clave pública e incluye códigos de corrección de errores. Mira más detalles aquí: https://unchained.com/blog/bitcoin-address-types-compared/

Vamos a crear un par de claves público y privada y una función que crea la descripción de una transferencia de nuestra cartera a otra cartera inventada



In [ ]:
# Clave de 2048 bits
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from base64 import b64encode
import json
from pprint import pprint

key = RSA.generate(1024)
mywallet = 'yNTNjNTBmYjg0YjQ3YTc2NzIxM2MxMjVjMj'
remote_wallet = 'MHhhZmViYzBlOGJkMTBlNzk5ZjI4NDI'

def transaction(key, mywallet, remote_wallet, amount, fee):
    info = dict(
        from_wallet = mywallet,
        to_wallet = remote_wallet,
        now = time.time(),
        amount=amount,
        fee=fee
    )
    signer = pkcs1_15.new(key)
    signature = b64encode(signer.sign(SHA256.new(data=json.dumps(info).encode()))).decode()
    return(dict(info=info, signature=signature))


pprint(transaction(key, mywallet, remote_wallet, 1, 0.1))

Este un ejemplo de una transacción real: https://blockchair.com/bitcoin/transaction/2749eaf2edc56b684c5f5e94ce6ca764cb82a2a3978033c36fffd8950275583a

Cuando necesitas hacer una transacción, creas una estructura de datos como esta y se la envías a un verificador/minero, que comprueba que la firma es correcta y que tienes fondos suficientes para realizar la transacción. Si todo va bien, se añade al pool de transacciones disponibles para crear un nuevo bloque.

## Bloques de bitcoin

Cuando un minero necesita crear un nuevo bloque: toma algunas de las transacciones que están esperando (si quieres que priorice la tuya... ¡págale más!) y las junta en un nuevo bloque

El nuevo bloque incluye:

- el hash del bloque anterior
- un conjunto de transacciones que están por meterse en un bloque
- la prueba de trabajo, usando un algoritmo similar a hashcat que veíamos antes. El esfuerzo de esta prueba de trabajo ha sido definido por todos los mineros de la red antes de ponerse a crear bloques.

![](images/blockchain.png)

Vamos a crear algunas transacciones y con ellas un nuevo bloque, usando un hash inventado como "el hash del bloque anterior"

In [ ]:
transactions = [
    transaction(key, mywallet, remote_wallet, 0.5, 0.1),
    transaction(key, mywallet, remote_wallet, 0.2, 0.1),
    transaction(key, mywallet, remote_wallet, 0.5, 0.1),
]
last_block_hash = '4145ba9c5ca6c55434c05ec080f07725fb282019e06ad127868f8954c1d6ad8'
effort = 5

block = dict(transactions=transactions, last_block_hash=last_block_hash)
proof = proof_of_work(json.dumps(block).encode(), effort)

pprint(proof)

Esto es lo que se llama minar un bloque. El primer minero que encuentra el contador, envía el bloque al mundo y empieza el proceso de nuevo

Mira un bloque real aquí: https://blockchair.com/bitcoin/block/798267

## Mecanismo de consenso

Cuando un bloque se acepta por los mineros, el siguiente bloque se creará incluyendo el valor de hash del antiguo. Así, los bloques están enlazados a través de sus valores de hash. Un bloque que ya esté en la blockchain no puede cambiar: cambiaría su hash, y el hash de todos los bloques que se metieron en la cadena después de él. Por eso, la importancia de la blockchain radica en que todos los mineros estén de acuerdo en qué bloques están dentro de la cadena ahora mismo.

Un bloque podría ser encontrado por varios mineros a la vez. Cuando un minero encuentra un nuevo bloque, lo envía al mundo y el resto de mineros puede aceptarlo o no. En realidad, es posible que mineros diferentes tengan opiniones diferentes sobre quién encontró primero un bloque, así que se inicia un sistema de consenso para decidir cuando la red encuentra este tipo de problemas y qué bloque es el siguiente que se tiene que meter en la cadena.

![](images/consensus.png)

El consenso es sencillo: si hay opiniones diferentes, se aceptan los bloques que más mineros hayan incorporado a su propia cadena. Puedes encontrar más información aquí https://en.bitcoin.it/wiki/Consensus o aquí https://notepub.io/notes/blockchain-technology/bitcoin/bitcoin-consensus-in-bitcoin/

## Cómo seguir

En este ejercicio hemos visto una simplificación de cómo funcione bitcoin. La implementación real tiene centenares de detalles adicionales. Por ejemplo: ¿cómo sabe alguien cuaĺ es el balance de una cartera? ¿cómo se evita gastar dos veces el mismo bitcoin?

Todos estos mecanismos ya no son criptográficos sino operativos. Para saber más, te animo a consultar:

- Cálculo del balance de una cartera: https://bitcoin.stackexchange.com/questions/22997/how-is-a-wallets-balance-computed
- Explicación de cómo funciona bitcoin: https://www.youtube.com/watch?v=bBC-nXj3Ng4
- Explicación alternativa: https://www.youtube.com/watch?v=Lx9zgZCMqXE
